In [5]:
# Import Libraries to get started

from pathlib import Path

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LassoCV, BayesianRidge
import statsmodels.formula.api as sm
import matplotlib.pylab as plt

import dmba
from dmba import regressionSummary, exhaustive_search
from dmba import backward_elimination, forward_selection, stepwise_selection
from dmba import adjusted_r2_score, AIC_score, BIC_score

%matplotlib inline

In [6]:
# Import each dataset seperately, assign to dataframes

df_customers = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/customers_final.csv')
df_engagement = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/engagements_final.csv')
df_marketing = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/marketing_final.csv')
df_transactions = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/transactions_final.csv')

In [32]:
df_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129545 entries, 0 to 129544
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   transaction_id      129545 non-null  int64  
 1   customer_id         129545 non-null  int64  
 2   transaction_date    129545 non-null  object 
 3   transaction_amount  129545 non-null  float64
 4   product_category    129545 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 4.9+ MB


In [13]:
# Create main df by merging all datasets

df = pd.merge(df_customers,df_engagement, on='customer_id', how='outer')

In [15]:
df = pd.merge(df,df_marketing,on='customer_id', how='outer')

In [17]:
df = pd.merge(df,df_transactions,on='customer_id', how='outer')

In [ ]:
# 'customer_id'
# 'join_date'
# 'last_purchase_date'
# 'age'
# 'gender'
# 'location'
# 'number_of_site_visits'
# 'number_of_emails_opened'
# 'number_of_clicks'
# 'campaign_id'
# 'response'
# 'promotion_type'
# 'campaign_date'
# 'transaction_id'
# 'transaction_date'
# 'transaction_amount'
# 'product_category'

In [20]:
df.columns

Index(['customer_id', 'join_date', 'last_purchase_date', 'age', 'gender',
       'location', 'number_of_site_visits', 'number_of_emails_opened',
       'number_of_clicks', 'campaign_id', 'response', 'promotion_type',
       'campaign_date', 'transaction_id', 'transaction_date',
       'transaction_amount', 'product_category'],
      dtype='object')

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327251 entries, 0 to 327250
Data columns (total 17 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   customer_id              327251 non-null  int64  
 1   join_date                327251 non-null  object 
 2   last_purchase_date       327251 non-null  object 
 3   age                      293971 non-null  float64
 4   gender                   309405 non-null  object 
 5   location                 327251 non-null  object 
 6   number_of_site_visits    327251 non-null  int64  
 7   number_of_emails_opened  327251 non-null  int64  
 8   number_of_clicks         327251 non-null  int64  
 9   campaign_id              327251 non-null  int64  
 10  response                 327251 non-null  object 
 11  promotion_type           327251 non-null  object 
 12  campaign_date            327251 non-null  object 
 13  transaction_id           327251 non-null  int64  
 14  tran

In [26]:
# Clean Dates

df['join_date'] = pd.to_datetime(df['join_date'])
df['last_purchase_date'] = pd.to_datetime(df['last_purchase_date'])
df['campaign_date'] = pd.to_datetime(df['campaign_date'])
df['transaction_date'] = pd.to_datetime(df['transaction_date'])


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327251 entries, 0 to 327250
Data columns (total 17 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   customer_id              327251 non-null  int64         
 1   join_date                327251 non-null  datetime64[ns]
 2   last_purchase_date       327251 non-null  datetime64[ns]
 3   age                      293971 non-null  float64       
 4   gender                   309405 non-null  object        
 5   location                 327251 non-null  object        
 6   number_of_site_visits    327251 non-null  int64         
 7   number_of_emails_opened  327251 non-null  int64         
 8   number_of_clicks         327251 non-null  int64         
 9   campaign_id              327251 non-null  int64         
 10  response                 327251 non-null  object        
 11  promotion_type           327251 non-null  object        
 12  campaign_date   

In [36]:
# Revenue by Year

revenue_yearly = df.groupby(df['transaction_date'].dt.year)['transaction_amount'].sum().reset_index()

revenue_yearly

,transaction_date,transaction_amount
0,2020,16114326.69
1,2021,35686959.68
2,2022,50564441.20
3,2023,62847073.46
4,2024,30017930.20
